# Sources

1. Kulldorff et al, "A Space–Time Permutation Scan Statistic for Disease Outbreak Detection", PLoS Med 2(3): e59, DOI:10.1371/journal.pmed.0020059
2. Kulldorff M. and Information Management Services, Inc. SaTScanTM v8.0: Software for the spatial and space-time scan statistics. http://www.satscan.org/, 2009.
3. Cheng, Adepeju, "Modifiable Temporal Unit Problem (MTUP) and Its Effect on Space-Time Cluster Detection", PLOS ONE 9(6): e100465. DOI:10.1371/journal.pone.0100465
4. Adepeju, Rosser, Cheng, "Novel evaluation metrics for sparse spatiotemporal point process hotspot predictions - a crime case study", International Journal of Geographical Information Science, 30:11, 2133-2154, DOI:10.1080/13658816.2016.1159684

# Discussion

The "Space-Time (Permutation) Scan Statistic" comes from a family of statistical tests for clustering in space-time point process data.  The freely available ("free as in beer"; source code is seemingly not available) software (2) implements a variety of cluster detection algorithms.  The particular algorithm we are interested in was first proposed in (1).

The basic idea is to look at the space-time data, and to consider all possible "cylinders": pick a location $(x,y)$ in space, a radius $r$ and a start and end time, $t_s, t_e$.  Then we consider all (space) points within distance $r$ of $(x,y)$, and all time points $t_s \leq t \leq t_e$ (visualised in three dimensions, this is indeed a cylinder).  We then have a choice of an underlying statistical model which allows us to compute the expected number of events in the cylinder.  We compare this expected number against the actual number of events which occurred, and single out the most "extreme" cylinder as our (primary) cluster.  Once done, we can then single out other cylinders as "secondary" clusters.

The user-manual (2) contains a large bibliography, including applications to crime analysis.  We shall follow (3) and (4) as they give fairly precise details of how the software (2) was used.  The website of (2) also lists other software, but none of these packages seem to implement the "permutation" algorithm discussed here.

# Derivation of the algorithm

We follow (1) but make some comments to allow us to treat data which is more "continuous".  Following (1), suppose we have that events can occur in a number of regions indexed by $z$ (for zip-code in the original) and in time regions indexed by $d$ (for days in the original).  Let $c_{z,d}$ be the count of events occurring in space region $z$ and time region $d$, so that the total number of events is
$$ N = \sum_{z,d} c_{z,d}. $$
We conditional on the marginals to obtain the expected number of events at $(z,d)$ as
$$ \mu_{z,d} = \frac{1}{N} \Big( \sum_{z'} c_{z',d} \Big) \Big( \sum_{d'} c_{z,d'} \Big). $$
The probabilistic assumption we have made here is that, if $e$ is a randomly chosen event from the $N$ total events, then the probability that $e$ is in region $z$, given it occurred in time region $d$, is the same for any choice of $d$ (and with the roles of $z$ and $d$ reversed).  If $A$ is some space-time region, then the expected number of events in region $A$ is
$$ \mu_A = \sum_{(z,d)\in A} \mu_{z,d}. $$
Our statistical test will be to compare the actual number of events which occur in $A$ against $\mu_A$, to detect regions where there is a significant departure from the expected number.

First, let us expand upon the definition of $\mu_A$.  In all cases we are interested in, $A$ will be a space-time "cylinder", meaning in particular that $A = B \times C$ where $B$ is some subset of space, and $C$ is some subset of time.  Then
$$ \mu_A = \mu_{B\times C} = \sum_{z\in B, d\in C} \mu_{z,d}
= \frac{1}{N} \Big( \sum_{z', d\in C} c_{z',d} \Big) \Big( \sum_{z\in B, d'} c_{z,d'} \Big). $$
In this form, it is easy to extend the definition to the case when the data is not "binned" into regions, but where we have point events occuring at space-time coordinates $(s_i,t_i)$ for $i=1,\ldots,N$:
$$ \mu_A = \frac{1}{N} \#\{ i : t_i\in C\} \#\{ i : s_i\in B \}, $$
where, for example, $\#\{ i : t_i\in C\}$ denotes the number of events which occur in time region $C$, and anywhere in space.

Continuing with following (1), we approximate, and consider the Poisson generalised likelihood ratio
$$ \Big(\frac{c_A}{\mu_A}\Big)^{c_A} \Big(\frac{N-c_A}{N-\mu_A}\Big)^{(N-c_A)} $$
Here $c_A$ denotes the number of events which actually occurred in space-time cylinder $A$, and $\mu_A$ is as before.  We search for the $A$ with this ratio being greatest.  (2) introduces a further factor, and multiplies by
$$ I(c_A > \mu_A) = \begin{cases} 1 : c_A > \mu_A, \\ 0 : c_A\leq \mu_A. \end{cases} $$
That is, we set the likelihood to be identically $0$ when there are fewer than expected events.  This ensures we single out only actual clusters, and not regions which are "non-clusters" (regions where there are many fewer than expected events).

We effectively test an infinite number of cylinders, and so simple hypothesis testing is not applicable (because of "multiple testing").  Instead we use Monte Carlo hypothesis testing: the idea here is to randomly generate data which has the same "distribution" as the real data, and to see how likely (or not) our statistic is.  In this case, we use the "permutation" from the title of (1).  We take the actual data, and then randomly shuffle the timestamps, while keeping the space locations to the same.  This does not change the marginal distributions.  For each such randomly chosen permutation, we re-calculate the likelihood ratio as above, finding the (possibly new) cylinder with the greatest value.  In this way we collect samples from the "typical" largest likelihood ratio, and then compare our actual value against this.  If we repeat this$n-1$ times, we can then rank the real data as being the $r$ th largest ratio.  Then the "$p$-value" of the cylinder is $r/n$.

To find the "most clustered" cylinder we find the cylinder with the largest statistic (i.e. largest value of the (log) generalised likelihood ratio).  Notice that this is the same as the cylinder with the smallest $p$-value.  (Which is useful, as it we are interested in prediction only, we may not need to actually estimate the $p$-value at all, thus saving a lot of computational time.  Of course, the $p$-value does give some indication of how much we might trust the prediction.)

Once done, we could find the next most extreme cylinder.  However, this will be essentially the same, as slightly deforming a cylinder will, in general, not change the likelihood (it will only change if the deformed cylinder contains more or fewer points, which won't happen for a sufficiently small change).  It is common then to only look at cylinders which are disjoint from the cylinders already singled out.  This is the approach taken in (3) (and presumably (4)).

An alternative, implemented in (2), is to delete the data points which occur in the previously identified clusters, and then re-run the entire process.  This can lead to overlapping cylinders, but these now have some statistical content-- as suggested in (2), we might expect a cluster in the very centre of city, and then a larger cluster formed from the "donut shape" of the inner urban area of a city, exluding the very centre.

Finally, a note on optimisations.  We obviously cannot consider all possible cylinders, as there are an infinite number.  As noted above, as both real and simulated data has the same space and time locations (though perhaps with the link between space and time "permuted") if two cylinders contain the same events, then they will always return the same likelihood.  Thus we need only consider cylinders which contain one point each, then cylinders which contain 2 points each, and so forth.  We also limit cylinders to not be above a certain size, or to contain too much of the data.

# The algorithm for grided data

We lay down a grid and assign events to grid cells.  When considering a "cylinder" of space-time, we consider intervals of time, and disks of space.  In space, the disks are always centred at the middle point of a grid cell, and when considering events, if the disk overlaps the centre of another cell, then all events in that cell are classed as being inside the disk.  As such, we only consider cells which are centred at the centre of grid cells, and have radius of $0$ (count only events in that cell) and then disks which exactly enclose some neighbouring cells.  To avoid complications from Pythagoras's theorem, we might instead just come up with some notion of "neighbouring cell" when considering "disks".  We do this in our implementation.

Following (1) and (3), we only consider disks which contain up to 50% of the events, or we limit the size of the disk (a maximum of 1000m is used in (3)).

In considering the intervals of time which will make up cylinders, we use the optimisation suggested by (4), and assume we are only interested in "prediction".  As such, we will take all data up to a time point $T_0$ which is the time "now", and will only be interested in clusters which extend up to the present time.  There is, however, a subtle point here.  Either we could perform our analysis by only considering cylinders which extend up to time $T_0$.  Or we could consider all cylinders, form the "primary" and then "secondary" clusters, and then finally only select the clusters which extend up to the time "now".  We will implement both ideas.

Let $N$ be the total number of events.  For a disk $D$ in space, and a time interval $I$, we compute $c_{D,I}$ the actual number of events which occurred in the disk and time interval, and compute $f(D)$, the number of events which occur in $D$ (at any time) and $g(I)$, the number of events which occur in time interval $I$ (at any location).  Then the expected number of events occurring is
$$ \mu_{D,I} = \frac{f(D) g(I)}{N} $$
If $c_{D,I} > \mu_{D,I}$ we set
$$ r_{D,I} = \log \Big[\Big(\frac{c_{D,I}}{\mu_{D,I}}\Big)^{c_{D,I}} \Big(\frac{N-c_{D,I}}{N-\mu_{D,I}}\Big)^{(N-c_{D,I})}\Big] 
= c_{D,I} \log \Big(\frac{c_{D,I}}{\mu_{D,I}}\Big) + (N-c_{D,I}) \log\Big(\frac{N-c_{D,I}}{N-\mu_{D,I}}\Big) $$
and otherwise we set $r_{D,I} = -\infty$.

We then maximise $r_{D,I}$ over all choices of $D,I$ top find our "cluster".

(If we are then interested in $p$-values, then $k-1$ times (typically $k=1000$) we randomly shuffle the timestamps of the data, and then recompute $r_{D,I}$ for all $D,I$, finding again the largest value.  We order, from greatest to lowest, the resulting ratios, and compute the rank of where the real data occurs, say $1 \leq r \leq k$, and compute the $p$-value as $p=r/k$.)

- Once done, we either then look at all _disjoint_ sets $D\times I$, and find the largest $r$ out of the remaining cylinders, continuing until we reach a cutoff of $p$ or exhaust the events.
- Alternatively, we delete the events occurring in the cluster, and then re-run the whole analysis.

We have already considered limiting the number of disks studied when we gridded the data.  We do not need to consider all possible time intervals, as both $c_{D,I}$ and $\mu_{D,I}$ depend only on the number of events in $D$, in $I$, and in both $D$ and $I$.

- Thus we need only consider enough intervals $I$ to distinguish the case when an event is in $I$ from when an event is not in $I$.
- Let the events have timestamps of $t_1 < t_2 < \cdots < t_N$ (if some events share timestamps, we deal with this by double counting, so perhaps we actually have fewer than $N$ distinct timestamps).
- If we are considering all intervals, we need to consider the intervals which contain just $t_1$, just $t_2$, and so on.  Then we consider the intervals which contain just $\{t_1,t_2\}$, just $\{t_2,t_3\}$, and so on and so forth.  This leads to $N + (N-1) + \cdots + 1 = N(N+1)/2$ choices.
- If we only wish to consider intervals up to the end of time, we only need consider the interval $[t_N, T_0]$, then $[t_{N-1}, T_0]$, and so on.  This gives $N$ choices in total.

